# 동적 계획법 (Dynamic Programming)
ch. 08 - 09  

중복되는 부분 문제 : 두 번 이상 계산되는 부분 문제  
ex) 점화식, 이항 계수의 계산  

두 번 이상 반복 계산되는 부분 문제의 답을 미리 저장하여 속도를 향상시킴  

메모이제이션을 적용할 수 있는 경우 : 참조적 투명 함수의 경우에만 가능  
참조적 투명성 : 함수의 반환 값이 그 입력 값만으로 결정되는지의 여부  
참조적 투명 함수 : 입력이 고정되어 있을 때 그 결과가 항상 같은 함수  

### 메모이제이션의 시간 복잡도 분석
(존재하는 부분 문제의 수) * (한 부분 문제를 풀 때 필요한 반복문의 횟수)  



In [1]:
# 재귀 호출을 이용한 이항 계수 계산
def bino(n, r):
    if r == 0 or n == r:
        return 1
    return bino(n-1, r-1) + bino(n-1, r)

print(bino(3,2))

3


In [5]:
# 메모이제이션을 이용한 이항 계수 계산
def bino2(n, r):
    memo = [[-1 for _ in range(r+1)] for _ in range(n+1)]
    if r == 0 or r == n:
        return 1
    if memo[n][r] != -1:
        return memo[n][r]
    memo[n][r] = bino2(n-1, r-1) + bino2(n-1, r)
    return memo[n][r]

print(bino2(18,2))

153


3 테스트 케이스의 수  
he?p 와일드카드 패턴  
3       파일명의 수  
help      n줄에 파일명  
heap  
helpp  
\*p\*  
3       파일명의 수  
help  
papa  
hello  
\*bb\*  
1        파일명의 수  
babbbc  

In [2]:
# 8.6 와일드 카드
def match(w, s):
    pos = 0
    while (pos<len(s)) and (pos<len(w)) and \
    (w[pos]=='?' or w[pos]==s[pos]):
        pos += 1
        
    if pos == len(w):
        return pos==len(s)
    
    if w[pos] == '*':
        for skip in range(len(s)-pos+1):
            if match(w[pos+1:], s[pos+skip:]):
                return True
    return False

print(match('he?p', 'help'))
print(match('he?p', 'heap'))
print(match('he?p', 'helpp'))

True
True
False


In [5]:
# 8.7 와일드카드 (DP)
# 캐시는 어떻게 초기화하는거지..?
cache = [ [-1 for _ in range(101)] for _ in range(101) ]

W = 'he?p'
S = 'helpp' # heap, helpp
def match(w, s):
    ret = cache[w][s]
    if ret != -1:
        return ret
    
    while (s<len(S)) and (w<len(W)) and \
    (W[w]=='?' or W[w]==S[s]):
        return match(w+1, s+1)
        
    if w == len(W):
        cache[w][s] = (s==len(S))
        return s==len(S)
    
    if W[w] == '*':
        if match(w+1, s) or (s<len(S) and match(w,s+1)):
            return True
    return False

print(match(0, 0))

False


In [24]:
# 문제 8.5 합친 LIS
import sys
neginf = - sys.maxsize + 1

def jlis(idx_a, idx_b):
    global A, B, n, m
    ret = cache[idx_a+1][idx_b+1]
    
    if ret != -1:
        return ret
    
    ret = 2
    if idx_a == -1:
        a = neginf
    else:
        a = A[idx_a]
        
    if idx_b == -1:
        b = neginf
    else:
        b = B[idx_b]
        
    max_element = max(a,b)
    
    for next_a in range(idx_a+1, n):
        if max_element < A[next_a]:
            ret = max(ret, jlis(next_a, idx_b)+1)
            
    for next_b in range(idx_b+1, m):
        if max_element < B[next_b]:
            ret = max(ret, jlis(idx_a, next_b)+1)
    
    cache[idx_a][idx_b] = ret
    return ret

n = 3
m = 3
A = [1,2,4]
B = [3,4,7]
cache = [ [-1 for _ in range(101)] for _ in range(101) ]
print(jlis(-1,-1)-2)

n = 3
m = 3
A = [1,2,3]
B = [4,5,6]
cache = [ [-1 for _ in range(101)] for _ in range(101) ]
print(jlis(-1,-1)-2)

n = 5
m = 3
A = [10,20,30,1,2]
B = [10,20,30]
cache = [ [-1 for _ in range(101)] for _ in range(101) ]
print(jlis(-1,-1)-2)

5
6
5


In [27]:
# 문제 8.12 비대칭 타일링
mod = 1000000007

def tiling(width):
    if width <= 1:
        return 1
    ret = cache[width]
    if ret != -1:
        return ret
    ret = (tiling(width-2) + tiling(width-1)) % mod
    cache[width] = ret
    return ret

def asymmetric(width):
    global mod
    if width % 2 == 1: # 홀수
        return (tiling(width) - tiling(width//2) + mod) % mod
    ret = tiling(width)
    ret = (ret - tiling(width//2) + mod) % mod # 절반이 대칭
    ret = (ret - tiling(width//2-1) + mod) % mod # 가운데 세로줄을 가로 타일이 덮는 경우
    return ret

n = [3, 2, 4, 92]
for i in n:
    cache = [-1 for _ in range(101)]
    print(asymmetric(i))

2
0
2
913227494


In [29]:
# 비대칭 타일 수를 세는 방법
mod = 1000000007

def asymmetric2(width):
    if width <= 2:
        return 0
    
    ret = cache2[width]
    if ret != -1:
        return ret
    
    ret = asymmetric2(width-2) % mod
    ret = (ret + asymmetric2(width-4)) % mod
    ret = (ret + tiling(width-3)) % mod
    ret = (ret + tiling(width-3)) % mod
    cache2[width] = ret
    return ret

n = [3, 2, 4, 92]
for i in n:
    cache2 = [-1 for _ in range(101)]
    print(asymmetric2(i))

2
0
2
913227494


In [36]:
# 문제 8.14 폴리오미노
mod = 10 * 1000 * 1000

def poly(n, first):
    if n == first:
        return 1
    
    ret = cache[n][first]
    
    if ret != -1:
        return ret
    
    ret = 0
    for second in range(1, n-first+1):
        add = second + first -1
        add *= poly(n-first, second)
        add %= mod
        ret += add
        ret %= mod

    cache[n][first] = ret
    return ret

num = [2, 4, 92]
for n in num:
    cache = [[-1 for _ in range(100)] for _ in range(100)]
    answer = 0
    for i in range(1, n+1):
        answer += poly(n, i)
    print(answer % mod)

2
19
4841817


In [41]:
# 문제 9.2 여행 짐 싸기
# W 보다는 capacity가 작으면서
# 절박도는 최대가 되도록 아이템 담기
def pack(capacity, item):
    if item == N:
        return 0
    
    ret = cache[capacity][item]
    
    if ret != -1:
        return ret
    
    ret = pack(capacity, item+1)
    
    if capacity >= volume[item]:
        ret = max(ret, pack(capacity-volume[item], item+1)+need[item])
        
    cache[capacity][item] = ret
    
    return ret # 최대 절박도

def reconstruct(capacity, item, picked):
    if item == N:
        return
    
    if pack(capacity, item) == pack(capacity, item+1):
        reconstruct(capacity, item+1, picked)
    else:
        picked.append(name[item])
        reconstruct(capacity-volume[item], item+1, picked)

# testcase 1
N, W = 6, 10
info = list()
info.append(["laptop", 4, 7])
info.append(["camera", 2, 10])
info.append(["xbox", 6, 6])
info.append(["grinder", 4, 7])
info.append(["dumbell", 2, 5])
info.append(["encyclopedia", 10, 4])
name = [x[0] for x in info]
volume = [x[1] for x in info]
need = [x[2] for x in info]
cache = [[-1 for _ in range(101)] for _ in range(101)]
picked = []
print(pack(W, 0))
reconstruct(W, 0, picked)
print(picked)
# testcase 2
N, W = 6, 17
info = list()
info.append(["laptop", 4, 7])
info.append(["camera", 2, 10])
info.append(["xbox", 6, 6])
info.append(["grinder", 4, 7])
info.append(["dumbell", 2, 5])
info.append(["encyclopedia", 10, 4])
name = [x[0] for x in info]
volume = [x[1] for x in info]
need = [x[2] for x in info]
cache = [[-1 for _ in range(101)] for _ in range(101)]
picked = []
print(pack(W, 0))
reconstruct(W, 0, picked)
print(picked)

24
['laptop', 'camera', 'grinder']
30
['laptop', 'camera', 'xbox', 'grinder']


In [ ]:
# 문제 9.9 드래곤 커브


In [ ]:
# 문제 9.14

In [ ]:
# 문제 9.19

In [ ]:
# 문제 9.22